In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']

In [2]:
path=r"C:\Users\admin\Desktop\RCPMF\A.csv"

In [3]:
df=pd.read_csv(path)

In [4]:
# (1) 处理日期数据
def split_month(x):
    return x[5:7]
df["S"]=df["Q"].apply(split_month)
df=df.drop_duplicates()
# (2) 添加季节特征
def season(x):
    if x=="01" or x=="02" or x=="12":
        return "冬"
    if x=="03" or x=="04" or x=="05":
        return "春"
    if x=="06" or x=="07" or x=="08":
        return "夏"
    if x=="09" or x=="10" or x=="11":
        return "秋"
df["Season"]=df["S"].apply(season)

In [5]:
X=df[['A', 'B', 'D', 'E', 'F', 'G','H', 'K', 'L', 'M', 'N', 'Q',"S","Season"]]
y=df[["R"]]

In [6]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.8,random_state=2)

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1 MV

In [7]:
# 训练集
X_train_Mean=pd.concat([X_train,y_train],axis=1)

In [8]:
X_train_Mean=X_train_Mean.drop_duplicates()

In [9]:
X_train_Mean_A=X_train_Mean[X_train_Mean["Season"]=="春"]
X_train_Mean_B=X_train_Mean[X_train_Mean["Season"]=="夏"]
X_train_Mean_C=X_train_Mean[X_train_Mean["Season"]=="秋"]
X_train_Mean_D=X_train_Mean[X_train_Mean["Season"]=="冬"]

In [10]:
X_test_Mean=pd.concat([X_test,y_test],axis=1)
X_test_Mean_A=X_test_Mean[X_test_Mean["Season"]=="春"]
X_test_Mean_B=X_test_Mean[X_test_Mean["Season"]=="夏"]
X_test_Mean_C=X_test_Mean[X_test_Mean["Season"]=="秋"]
X_test_Mean_D=X_test_Mean[X_test_Mean["Season"]=="冬"]

In [11]:
# 春季用户列表
user_A=X_train_Mean_A["A"].unique()
item_A=X_train_Mean_A["B"].unique()
# 夏季用户列表
user_B=X_train_Mean_B["A"].unique()
item_B=X_train_Mean_B["B"].unique()
# 秋季用户列表
user_C=X_train_Mean_C["A"].unique()
item_C=X_train_Mean_C["B"].unique()
# 冬季用户列表
user_D=X_train_Mean_D["A"].unique()
item_D=X_train_Mean_D["B"].unique()

In [12]:
# 遍历测试集
X_test_Mean_A=X_test_Mean_A[["A","B","R"]]
X_test_Mean_B=X_test_Mean_B[["A","B","R"]]
X_test_Mean_C=X_test_Mean_C[["A","B","R"]]
X_test_Mean_D=X_test_Mean_D[["A","B","R"]]

In [13]:
# 遍历测试集
def Mean_Predict(X_test,X_train,users,items):
    result_mean=[]
    for user,item,rui in X_test.as_matrix():
        if user not in users:
            continue
        if item not in items:
            continue
        # 判断用户与物品是否存在于训练集中
        if len(X_train[(X_train["A"]==user) &(X_train["B"]==item)])<2:
            continue
        else:
            predict=X_train[(X_train["A"]==user) &(X_train["B"]==item)]["R"].mean()
            result_mean.append([user,item,rui,predict])
    return result_mean

In [14]:
X_test_A=Mean_Predict(X_test_Mean_A,X_train_Mean,user_A,item_A)
X_test_B=Mean_Predict(X_test_Mean_B,X_train_Mean,user_B,item_B)
X_test_C=Mean_Predict(X_test_Mean_C,X_train_Mean,user_C,item_C)
X_test_D=Mean_Predict(X_test_Mean_D,X_train_Mean,user_D,item_D)

In [15]:
X_test_A=pd.DataFrame(X_test_A)
X_test_A=X_test_A.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

In [16]:
X_test_B=pd.DataFrame(X_test_B)
X_test_B=X_test_B.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_C=pd.DataFrame(X_test_C)
X_test_C=X_test_C.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_D=pd.DataFrame(X_test_D)
X_test_D=X_test_D.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

In [17]:
X_test_A[:3]

,用户,物品,真实值,平均预测值
0,蜂锦,CX150833J;,17,45.0
1,环富,KX151140J;,34,35.5
2,环富,TX160843J;,45,62.0


In [18]:
# rmse
def RMSE(deviation):
    sum=0.0
    for i in deviation:
        sum+=i*i
    rmse=np.sqrt(sum/len(deviation))
    return rmse

In [19]:
# rmse
def RMSM_Mean(r1,r2,r3,r4):
    rmse=0.0
    rmse=(r1+r2+r3+r4)/4
    return rmse

In [20]:
# 春 LR+UPMF
RMSE(X_test_A["平均预测值"]-X_test_A["真实值"])

18.141306344493874

In [21]:
RMSE(X_test_B["平均预测值"]-X_test_B["真实值"])

105.66571250104695

In [22]:
RMSE(X_test_C["平均预测值"]-X_test_C["真实值"])

19.876004516893115

In [23]:
RMSE(X_test_D["平均预测值"]-X_test_D["真实值"])

37.5665828649366

In [24]:
RMSM_Mean(RMSE(X_test_A["平均预测值"]-X_test_A["真实值"]),RMSE(X_test_B["平均预测值"]-X_test_B["真实值"]),RMSE(X_test_C["平均预测值"]-X_test_C["真实值"]),RMSE(X_test_D["平均预测值"]-X_test_D["真实值"]))

45.31240155684264

# 2 MA

In [36]:
X=df[["A","B","Q","Season"]]

In [37]:
y=df["R"]

In [38]:
X_train_move,X_test_move,y_train_move,y_test_move=train_test_split(X,y,test_size=0.8,random_state=2)

In [39]:
# 合并训练集数据
X_train_Move_Mean=pd.concat([X_train_move,y_train_move],axis=1)

In [40]:
# 根据时间排序
X_train_Move_Mean.sort_values(by=["Q"],inplace=True)

In [41]:
X_train_Move_Mean_A=X_train_Move_Mean[X_train_Move_Mean["Season"]=="春"]
X_train_Move_Mean_B=X_train_Move_Mean[X_train_Move_Mean["Season"]=="夏"]
X_train_Move_Mean_C=X_train_Move_Mean[X_train_Move_Mean["Season"]=="秋"]
X_train_Move_Mean_D=X_train_Move_Mean[X_train_Move_Mean["Season"]=="冬"]

In [42]:
# 合并测试集数据
X_test_Move_Mean=pd.concat([X_test_move,y_test_move],axis=1)
# 根据时间排序
X_test_Move_Mean.sort_values(by=["Q"],inplace=True)

In [44]:
X_test_Move_Mean_A=X_test_Move_Mean[X_test_Move_Mean["Season"]=="春"]
X_test_Move_Mean_B=X_test_Move_Mean[X_test_Move_Mean["Season"]=="夏"]
X_test_Move_Mean_C=X_test_Move_Mean[X_test_Move_Mean["Season"]=="秋"]
X_test_Move_Mean_D=X_test_Move_Mean[X_test_Move_Mean["Season"]=="冬"]

In [45]:
# 获取满足一定记录的数据记录
def get_recored(X_test,X_train):
    temp=pd.DataFrame(columns=["A","B","Q","Season","R"])
    for user,item,time,season,rui in X_test.as_matrix():
        if len(X_train[(X_train["A"]==user) & (X_train["B"]==item)])>=2:
            #print(X_move_mean_train[(X_move_mean_train["供应商"]==user) & (X_move_mean_train["订单款号"]==item)])
            t=X_train[(X_train["A"]==user) & (X_train["B"]==item)]
            temp=pd.concat([temp,t],axis=0)
    return temp
    
        
        

In [46]:
# 测试集存在于训练集中的用户物品且满足一定记录的数据集
temp_A=get_recored(X_test_Move_Mean_A,X_train_Move_Mean)
temp_B=get_recored(X_test_Move_Mean_B,X_train_Move_Mean)
temp_C=get_recored(X_test_Move_Mean_C,X_train_Move_Mean)
temp_D=get_recored(X_test_Move_Mean_D,X_train_Move_Mean)

In [47]:
# 假设根据最近2个记录最为最新的时间预测
def Move_Mean_Predict(X_test,X_train):
    result_move_mean=[]
    for user,item,time,season,rui in X_test.as_matrix():
        if len(X_train[(X_train["A"]==user)&(X_train["B"]==item)])>=2:
            value=X_train[(X_train["A"]==user)&(X_train["B"]==item)].sort_values(by=["Q"])[-2:]["R"].mean()
            result_move_mean.append([user,item,value,rui])
    return result_move_mean

In [48]:
X_test_Move_Mean_A=Move_Mean_Predict(X_test_Move_Mean_A,temp_A)
X_test_Move_Mean_B=Move_Mean_Predict(X_test_Move_Mean_B,temp_B)
X_test_Move_Mean_C=Move_Mean_Predict(X_test_Move_Mean_C,temp_C)
X_test_Move_Mean_D=Move_Mean_Predict(X_test_Move_Mean_D,temp_D)

In [49]:
X_test_Move_Mean_A=pd.DataFrame(X_test_Move_Mean_A)
X_test_Move_Mean_A=X_test_Move_Mean_A.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Move_Mean_B=pd.DataFrame(X_test_Move_Mean_B)
X_test_Move_Mean_B=X_test_Move_Mean_B.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Move_Mean_C=pd.DataFrame(X_test_Move_Mean_C)
X_test_Move_Mean_C=X_test_Move_Mean_C.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Move_Mean_D=pd.DataFrame(X_test_Move_Mean_D)
X_test_Move_Mean_D=X_test_Move_Mean_D.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

In [50]:
# rmse
def RMSE(deviation):
    sum=0.0
    for i in deviation:
        sum+=i*i
    rmse=np.sqrt(sum/len(deviation))
    return rmse

In [51]:
RMSM_Mean(RMSE(X_test_Move_Mean_A["平均预测值"]-X_test_Move_Mean_A["真实值"]),RMSE(X_test_Move_Mean_B["平均预测值"]-X_test_Move_Mean_B["真实值"]),RMSE(X_test_Move_Mean_C["平均预测值"]-X_test_Move_Mean_C["真实值"]),RMSE(X_test_Move_Mean_D["平均预测值"]-X_test_Move_Mean_D["真实值"]))

64.93685479247395

# 3 ES

In [52]:
X=df[["A","B","Q","Season"]]
y=df["R"]

In [53]:
X_train_index,X_test_index,y_train_index,y_test_index=train_test_split(X,y,test_size=0.8,random_state=2)

In [54]:
# 合并训练集数据
X_train_Index_Smooth=pd.concat([X_train_index,y_train_index],axis=1)
# 根据时间排序
X_train_Index_Smooth.sort_values(by=["Q"],inplace=True)

In [55]:
X_train_Index_Smooth_A=X_train_Index_Smooth[X_train_Index_Smooth["Season"]=="春"]
X_train_Index_Smooth_B=X_train_Index_Smooth[X_train_Index_Smooth["Season"]=="夏"]
X_train_Index_Smooth_C=X_train_Index_Smooth[X_train_Index_Smooth["Season"]=="秋"]
X_train_Index_Smooth_D=X_train_Index_Smooth[X_train_Index_Smooth["Season"]=="冬"]

In [56]:
# 合并测试集数据
X_test_Index_Smooth=pd.concat([X_test_index,y_test_index],axis=1)
# 根据时间排序
X_test_Index_Smooth.sort_values(by=["Q"],inplace=True)

In [57]:
X_test_Index_Smooth_A=X_test_Index_Smooth[X_test_Index_Smooth["Season"]=="春"]
X_test_Index_Smooth_B=X_test_Index_Smooth[X_test_Index_Smooth["Season"]=="夏"]
X_test_Index_Smooth_C=X_test_Index_Smooth[X_test_Index_Smooth["Season"]=="秋"]
X_test_Index_Smooth_D=X_test_Index_Smooth[X_test_Index_Smooth["Season"]=="冬"]

In [58]:
# 测试集存在于训练集中的用户物品且满足一定记录的数据集
temp_Index_A=get_recored(X_test_Index_Smooth_A,X_train_Index_Smooth)
temp_Index_B=get_recored(X_test_Index_Smooth_B,X_train_Index_Smooth)
temp_Index_C=get_recored(X_test_Index_Smooth_C,X_train_Index_Smooth)
temp_Index_D=get_recored(X_test_Index_Smooth_D,X_train_Index_Smooth)

In [59]:
# 假设根据最近2个记录最为最新的时间预测
def Index_Smooth_Predict(X_test,X_train):
    result_index_smooth=[]
    for user,item,time,season,rui in X_test.as_matrix():
        if len(X_train[(X_train["A"]==user)&(X_train["B"]==item)])>=2:
            values=X_train[(X_train["A"]==user)&(X_train["B"]==item)].sort_values(by=["Q"])["R"].values
            temp=values[0]  # 表示上一次预测的平滑值
            temp_predict=0.0
            for i in values:
                temp_predict=0.5*i+(1-0.5)*temp
                temp=temp_predict
            result_index_smooth.append([user,item,rui,temp_predict])
    return result_index_smooth

In [60]:
X_test_Index_Smooth_A=Index_Smooth_Predict(X_test_Index_Smooth_A,temp_Index_A)
X_test_Index_Smooth_B=Index_Smooth_Predict(X_test_Index_Smooth_B,temp_Index_B)
X_test_Index_Smooth_C=Index_Smooth_Predict(X_test_Index_Smooth_C,temp_Index_C)
X_test_Index_Smooth_D=Index_Smooth_Predict(X_test_Index_Smooth_D,temp_Index_D)

In [61]:
X_test_Index_Smooth_A=pd.DataFrame(X_test_Index_Smooth_A)
X_test_Index_Smooth_A=X_test_Index_Smooth_A.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Index_Smooth_B=pd.DataFrame(X_test_Index_Smooth_B)
X_test_Index_Smooth_B=X_test_Index_Smooth_B.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Index_Smooth_C=pd.DataFrame(X_test_Index_Smooth_C)
X_test_Index_Smooth_C=X_test_Index_Smooth_C.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

X_test_Index_Smooth_D=pd.DataFrame(X_test_Index_Smooth_D)
X_test_Index_Smooth_D=X_test_Index_Smooth_D.rename(columns={0: "用户",1: "物品",2: "真实值",3: "平均预测值"})

In [62]:
RMSM_Mean(RMSE(X_test_Index_Smooth_A["平均预测值"]-X_test_Index_Smooth_A["真实值"]),RMSE(X_test_Index_Smooth_B["平均预测值"]-X_test_Index_Smooth_B["真实值"]),RMSE(X_test_Index_Smooth_C["平均预测值"]-X_test_Index_Smooth_C["真实值"]),RMSE(X_test_Index_Smooth_D["平均预测值"]-X_test_Index_Smooth_D["真实值"]))

54.223997556034256

# 4 KNN法

In [73]:
X=df[['A', 'B', 'D', 'E','F', 'G', 'H','K', 'L', 'M', 'N', 'Q',"S","Season"]]
y=df[["R"]]

In [74]:
X_train_knn,X_test_knn,y_train_knn,y_test_knn=train_test_split(X,y,test_size=0.8,random_state=2)

In [75]:
X_train_knn=X_train_knn[['D','E','F','G','H','K' ,'L','M', 'N']]

In [76]:
# 使用KNN进行回归
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(5, weights="uniform")

In [77]:
knn.fit(X_train_knn, y_train_knn)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [78]:
X_test_knn_knn=X_test_knn[['D','E','F','G','H','K' ,'L','M', 'N']]

In [79]:
# 使用测试集进行预测
y_test_knn_predict=knn.predict(X_test_knn_knn)

In [80]:
y_test_knn_copy=y_test_knn.copy()

In [81]:
y_test_knn_copy["预测值"]=y_test_knn_predict

In [82]:
y_test_knn_copy["Season"]=X_test_knn["Season"]

In [83]:
X_test_KNN_A=y_test_knn_copy[y_test_knn_copy["Season"]=="春"]
X_test_KNN_B=y_test_knn_copy[y_test_knn_copy["Season"]=="夏"]
X_test_KNN_C=y_test_knn_copy[y_test_knn_copy["Season"]=="秋"]
X_test_KNN_D=y_test_knn_copy[y_test_knn_copy["Season"]=="冬"]

In [84]:
RMSM_Mean(RMSE(X_test_KNN_A["预测值"]-X_test_KNN_A["R"]),RMSE(X_test_KNN_B["预测值"]-X_test_KNN_B["R"]),RMSE(X_test_KNN_C["预测值"]-X_test_KNN_C["R"]),RMSE(X_test_KNN_D["预测值"]-X_test_KNN_D["R"]))

31.612825827374017